# BTC Prediction (Data Wrangling Stage)

### This notebook contains:
- Importing datasets (data collection via Binance exchange api) 
- Data cleaning and organizing (removal of NaN values)
- Data joining of Twitter sentiment and BTC FUT dataset
- Data joining and calculating of technical indicators (EMAs, ATR, ADX, and OBV)

In [1]:
import requests        
import json            
import pandas as pd    
import numpy as np     

import datetime 

import os # for api keys

from binance.client import Client # import binance client



In [2]:
# bring in api keys from system

api_key = os.environ.get('binance_api')
api_secret = os.environ.get('binance_secret')

In [3]:
# initiate client with apis

client = Client(api_key, api_secret)

In [4]:
from binance.websockets import BinanceSocketManager
from twisted.internet import reactor

In [5]:
# pull data in from Binance API

btc_hist = client.get_historical_klines("BTCUSDT", Client.KLINE_INTERVAL_1DAY, "1 Jul, 2019")

In [6]:
#check start and end timestamps 

btc_hist_range=np.array(btc_hist)
range_times = btc_hist_range[:,0]
start = int(range_times[1])
end = int(range_times[-1])
print(start, end)

1562025600000 1606694400000


In [7]:
print("start:", datetime.datetime.fromtimestamp(start/1000.0))
print("end:", datetime.datetime.fromtimestamp(end/1000.0))

start: 2019-07-01 20:00:00
end: 2020-11-29 19:00:00


In [8]:
type(btc_hist)

list

In [9]:
# create dataframe from Binance list data

btc_df = pd.DataFrame(btc_hist, columns=['openTime', 'open', 'high', 'low', 'close', 'volume', 'closeTime', 'quoteAssetVolume', 'numTrade', 'takerBuyBaseAssetVolume', 'takerByQuoteAssetVolume', 'ignore'])

## Selecting targeted columns

#### This project will only focus on the assets attributes:
- Close Time
- Open Price
- Session High
- Session Low
- Close Price
- Volume

Because we are only focusing on those 6 qualities of the trading day, we can drop the other columned atrributes like "quote asset volume"

In [10]:
# drop columns that are of no use

btc_df.drop(['openTime', 'quoteAssetVolume', 'numTrade', 'takerBuyBaseAssetVolume', 'takerByQuoteAssetVolume', 'ignore'], axis=1, inplace=True)

In [11]:
# rename columns

btc_df = btc_df[['closeTime', 'open', 'high', 'low', 'close', 'volume']]

In [12]:
# convert "closeTime" to readable date from miliseconds

btc_df['closeTime'] = pd.to_datetime(btc_df['closeTime'], unit='ms')

In [13]:
# drop time sequence from date in "closeTime"

btc_df['closeTime'] = pd.to_datetime(btc_df['closeTime']).dt.date

In [14]:
# set index to "closeTime"

btc_df.set_index(['closeTime'], inplace=True)

In [15]:
# rename index "closeTime" to "Date"

btc_df.index.names = ['Date']

In [16]:
# convert columns from str to float (for indicators)

btc_df[['open','high', 'low', 'close', 'volume']]=btc_df[['open','high', 'low', 'close', 'volume']].astype(float)

In [17]:
btc_df.head()

,open,high,low,close,volume
Date,,,,,
2019-07-01,10854.10,11282.28,10030.00,10624.93,90962.268271
2019-07-02,10624.90,10938.75,9727.00,10842.85,109561.038728
2019-07-03,10844.98,11991.89,10841.04,11940.00,96815.900290
2019-07-04,11940.00,12000.00,11055.00,11145.67,66512.221892
2019-07-05,11145.67,11406.83,10796.44,10970.73,63534.350582


## Technical Indicator Calculations and Joining to Dataframe

#### The indicators being added are:
- RSI (Relative Strength Index)
- EMA LONG (Exponential Moving Average with Long Period)
- EMA SHORT (Exponential Moving Average with Short Period)
- ATR (Average True Range)
- ADX (Average Directional Movement Index)
- OBV (On-Balance Volume)

The indicators are calculated using the BTAlib, using the historical price data contained in dataframe. 

In [18]:
# import technical indicator library 

import btalib

In [19]:
# create RSI indicator

rsi = btalib.rsi(btc_df, period=14)

# join RSI to dataframe

btc_df = btc_df.join([rsi.df])

In [20]:
# create EMA long version and rename to "ema_long"

ema_long = btalib.ema(btc_df.close, period=50)
ema_long.df.rename(columns={'ema': 'ema_long'}, inplace=True)

# create EMA short version and rename to "ema_short"

ema_short = btalib.ema(btc_df.close, period=20)
ema_short.df.rename(columns={'ema': 'ema_short'}, inplace=True)

# join ema_long and ema_short to dataframe

btc_df = btc_df.join([ema_long.df, ema_short.df])

In [21]:
# create ATR indicator

atr = btalib.atr(btc_df, period=14)

# create ADX indicator

adx = btalib.adx(btc_df, period=14)

# join ATR and ADX to dataframe

btc_df = btc_df.join([atr.df, adx.df])

In [22]:
# create OBV indicator

obv = btalib.obv(btc_df)

# join OBV to indicator

btc_df = btc_df.join([obv.df])

In [23]:
btc_df.head()

,open,high,low,close,volume,rsi,ema_long,ema_short,atr,adx,obv
Date,,,,,,,,,,,
2019-07-01,10854.10,11282.28,10030.00,10624.93,90962.268271,NaN,NaN,NaN,NaN,NaN,NaN
2019-07-02,10624.90,10938.75,9727.00,10842.85,109561.038728,NaN,NaN,NaN,NaN,NaN,109561.038728
2019-07-03,10844.98,11991.89,10841.04,11940.00,96815.900290,NaN,NaN,NaN,NaN,NaN,206376.939018
2019-07-04,11940.00,12000.00,11055.00,11145.67,66512.221892,NaN,NaN,NaN,NaN,NaN,139864.717126
2019-07-05,11145.67,11406.83,10796.44,10970.73,63534.350582,NaN,NaN,NaN,NaN,NaN,76330.366544


In [24]:
# create "prediction target" column (shift new column values up by 1)

btc_df["close_nextday"] = btc_df.close.shift(-1)

In [25]:
btc_df.head()

,open,high,low,close,volume,rsi,ema_long,ema_short,atr,adx,obv,close_nextday
Date,,,,,,,,,,,,
2019-07-01,10854.10,11282.28,10030.00,10624.93,90962.268271,NaN,NaN,NaN,NaN,NaN,NaN,10842.85
2019-07-02,10624.90,10938.75,9727.00,10842.85,109561.038728,NaN,NaN,NaN,NaN,NaN,109561.038728,11940.00
2019-07-03,10844.98,11991.89,10841.04,11940.00,96815.900290,NaN,NaN,NaN,NaN,NaN,206376.939018,11145.67
2019-07-04,11940.00,12000.00,11055.00,11145.67,66512.221892,NaN,NaN,NaN,NaN,NaN,139864.717126,10970.73
2019-07-05,11145.67,11406.83,10796.44,10970.73,63534.350582,NaN,NaN,NaN,NaN,NaN,76330.366544,11256.49


#### Since the technical indicators utilize an excess number of historical data in order to make the appropriate calculation for each trading day, there will be NaN values left over. Those NaN values are not within our targeted time range so we need to remove all days containing empty cells. 

In [26]:
# drop unecessary rows not in project time range (also removes NaN values created by indicators)

btc_df.drop(btc_df.head(63).index,inplace=True)

In [27]:
btc_df.head()

,open,high,low,close,volume,rsi,ema_long,ema_short,atr,adx,obv,close_nextday
Date,,,,,,,,,,,,
2019-09-02,9723.59,10450.00,9712.50,10340.00,44740.248093,51.381478,10452.265343,10164.518939,530.693553,19.114750,225053.863244,10615.28
2019-09-03,10340.00,10773.00,10272.00,10615.28,47998.376781,54.904081,10458.658074,10207.448563,528.572585,18.426961,273052.240025,10567.02
2019-09-04,10611.85,10799.00,10369.89,10567.02,43943.889026,54.163174,10462.907561,10241.693462,521.468114,17.842351,229108.350999,10564.49
2019-09-05,10565.92,10900.00,10450.00,10564.49,33970.960639,54.121941,10466.891187,10272.435990,516.363249,17.516155,195137.390360,10298.73
2019-09-06,10563.13,10905.87,10150.00,10298.73,58799.640959,49.830575,10460.296630,10274.940181,533.470874,16.423958,136337.749401,10455.88


In [28]:
# bring BTC futures prices

btc_fut = pd.read_csv("closeFUT.csv", index_col='Date', parse_dates=True)

In [29]:
btc_fut.shape

(443, 1)

In [30]:
btc_df.shape

(456, 12)

In [31]:
# join btc_fut dataframe to original btc_df

btc_df = btc_df.join(btc_fut, how="inner")

In [32]:
btc_df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 443 entries, 2019-09-02 to 2020-11-17
Data columns (total 13 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   open           443 non-null    float64
 1   high           443 non-null    float64
 2   low            443 non-null    float64
 3   close          443 non-null    float64
 4   volume         443 non-null    float64
 5   rsi            443 non-null    float64
 6   ema_long       443 non-null    float64
 7   ema_short      443 non-null    float64
 8   atr            443 non-null    float64
 9   adx            443 non-null    float64
 10  obv            443 non-null    float64
 11  close_nextday  443 non-null    float64
 12  closeFUT       443 non-null    float64
dtypes: float64(13)
memory usage: 48.5 KB


In [33]:
btc_df.head()

,open,high,low,close,volume,rsi,ema_long,ema_short,atr,adx,obv,close_nextday,closeFUT
Date,,,,,,,,,,,,,
2019-09-02,9723.59,10450.00,9712.50,10340.00,44740.248093,51.381478,10452.265343,10164.518939,530.693553,19.114750,225053.863244,10615.28,10341.07
2019-09-03,10340.00,10773.00,10272.00,10615.28,47998.376781,54.904081,10458.658074,10207.448563,528.572585,18.426961,273052.240025,10567.02,10610.72
2019-09-04,10611.85,10799.00,10369.89,10567.02,43943.889026,54.163174,10462.907561,10241.693462,521.468114,17.842351,229108.350999,10564.49,10568.07
2019-09-05,10565.92,10900.00,10450.00,10564.49,33970.960639,54.121941,10466.891187,10272.435990,516.363249,17.516155,195137.390360,10298.73,10565.04
2019-09-06,10563.13,10905.87,10150.00,10298.73,58799.640959,49.830575,10460.296630,10274.940181,533.470874,16.423958,136337.749401,10455.88,10294.96


In [34]:
# round volume column

btc_df['volume'] = btc_df['volume'].round(2)

In [35]:
btc_df.head()

,open,high,low,close,volume,rsi,ema_long,ema_short,atr,adx,obv,close_nextday,closeFUT
Date,,,,,,,,,,,,,
2019-09-02,9723.59,10450.00,9712.50,10340.00,44740.25,51.381478,10452.265343,10164.518939,530.693553,19.114750,225053.863244,10615.28,10341.07
2019-09-03,10340.00,10773.00,10272.00,10615.28,47998.38,54.904081,10458.658074,10207.448563,528.572585,18.426961,273052.240025,10567.02,10610.72
2019-09-04,10611.85,10799.00,10369.89,10567.02,43943.89,54.163174,10462.907561,10241.693462,521.468114,17.842351,229108.350999,10564.49,10568.07
2019-09-05,10565.92,10900.00,10450.00,10564.49,33970.96,54.121941,10466.891187,10272.435990,516.363249,17.516155,195137.390360,10298.73,10565.04
2019-09-06,10563.13,10905.87,10150.00,10298.73,58799.64,49.830575,10460.296630,10274.940181,533.470874,16.423958,136337.749401,10455.88,10294.96


### Adding Twitter Tweet Sentiment

The sentiment regarding BTCs price was calculated in a seperate notebook. However, the tweets for each day were only able to be compiled missing the last two days within the target time range. In order to match the data accordingly, we have to remove 2 days from the original BTC dataframe. 

The sentiment was gathered using mean polarity from TextBlobs. Each tweet for each day that mentioned "BTC" was given either the characteristic of positive or negative. Polarity being the degree of emotion expressed through the tweet. 

- A negative polarity is anything less than 0. 
- A polarity equal to 0 is neutral. 
- A positive polarity is more than 0. 

Positive or negative can be seen as either "bullish" or "bearish" in this projects case.

In [36]:
# drop 2 tail rows to fit tweet sentiment dataframe and match time range

btc_df.drop(btc_df.tail(2).index,inplace=True)

In [37]:
btc_df.tail()

,open,high,low,close,volume,rsi,ema_long,ema_short,atr,adx,obv,close_nextday,closeFUT
Date,,,,,,,,,,,,,
2020-11-11,15297.21,15965.00,15272.68,15684.24,78469.75,73.450202,12927.689795,14236.176564,698.560946,53.701439,1.954724e+06,16291.86,15685.02
2020-11-12,15684.25,16340.70,15440.64,16291.86,102196.36,76.977504,13059.618039,14431.955939,712.953736,53.845549,2.056921e+06,16320.70,16289.98
2020-11-13,16291.85,16480.00,15952.35,16320.70,75691.88,77.132794,13187.503606,14611.836326,699.717755,54.091743,2.132613e+06,16070.45,16321.89
2020-11-14,16320.04,16326.99,15670.00,16070.45,59116.35,72.559314,13300.560327,14750.751914,696.665772,53.511714,2.073496e+06,15957.00,16066.20
2020-11-15,16069.56,16180.00,15774.72,15957.00,43596.84,70.517941,13404.734432,14865.632684,675.852503,52.973116,2.029899e+06,16713.57,15952.73


In [38]:
# bring in tweet sentiment dataframe

tweet_sentiment_df = pd.read_csv('btc_sentiment.csv', index_col='date', parse_dates=True)

In [39]:
tweet_sentiment_df.head()

,tweet_sentiment
date,
2019-09-02,-1.0
2019-09-03,0.5
2019-09-04,0.5
2019-09-05,0.5
2019-09-06,0.0


In [40]:
# join tweet sentiment dataframe with btc_df

btc_df = btc_df.join(tweet_sentiment_df, how="inner")

In [41]:
btc_df.head()

,open,high,low,close,volume,rsi,ema_long,ema_short,atr,adx,obv,close_nextday,closeFUT,tweet_sentiment
Date,,,,,,,,,,,,,,
2019-09-02,9723.59,10450.00,9712.50,10340.00,44740.25,51.381478,10452.265343,10164.518939,530.693553,19.114750,225053.863244,10615.28,10341.07,-1.0
2019-09-03,10340.00,10773.00,10272.00,10615.28,47998.38,54.904081,10458.658074,10207.448563,528.572585,18.426961,273052.240025,10567.02,10610.72,0.5
2019-09-04,10611.85,10799.00,10369.89,10567.02,43943.89,54.163174,10462.907561,10241.693462,521.468114,17.842351,229108.350999,10564.49,10568.07,0.5
2019-09-05,10565.92,10900.00,10450.00,10564.49,33970.96,54.121941,10466.891187,10272.435990,516.363249,17.516155,195137.390360,10298.73,10565.04,0.5
2019-09-06,10563.13,10905.87,10150.00,10298.73,58799.64,49.830575,10460.296630,10274.940181,533.470874,16.423958,136337.749401,10455.88,10294.96,0.0


In [42]:
# reorder columns

btc_df = btc_df[['open', 'high', 'low', 'close', 'closeFUT', 'volume', 'rsi', 'ema_short', 'ema_long', 'atr', 'adx', 'obv', 'tweet_sentiment', 'close_nextday']]

In [43]:
btc_df.head()

,open,high,low,close,closeFUT,volume,rsi,ema_short,ema_long,atr,adx,obv,tweet_sentiment,close_nextday
Date,,,,,,,,,,,,,,
2019-09-02,9723.59,10450.00,9712.50,10340.00,10341.07,44740.25,51.381478,10164.518939,10452.265343,530.693553,19.114750,225053.863244,-1.0,10615.28
2019-09-03,10340.00,10773.00,10272.00,10615.28,10610.72,47998.38,54.904081,10207.448563,10458.658074,528.572585,18.426961,273052.240025,0.5,10567.02
2019-09-04,10611.85,10799.00,10369.89,10567.02,10568.07,43943.89,54.163174,10241.693462,10462.907561,521.468114,17.842351,229108.350999,0.5,10564.49
2019-09-05,10565.92,10900.00,10450.00,10564.49,10565.04,33970.96,54.121941,10272.435990,10466.891187,516.363249,17.516155,195137.390360,0.5,10298.73
2019-09-06,10563.13,10905.87,10150.00,10298.73,10294.96,58799.64,49.830575,10274.940181,10460.296630,533.470874,16.423958,136337.749401,0.0,10455.88


In [44]:
# check for any missing columns

btc_df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 441 entries, 2019-09-02 to 2020-11-15
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   open             441 non-null    float64
 1   high             441 non-null    float64
 2   low              441 non-null    float64
 3   close            441 non-null    float64
 4   closeFUT         441 non-null    float64
 5   volume           441 non-null    float64
 6   rsi              441 non-null    float64
 7   ema_short        441 non-null    float64
 8   ema_long         441 non-null    float64
 9   atr              441 non-null    float64
 10  adx              441 non-null    float64
 11  obv              441 non-null    float64
 12  tweet_sentiment  441 non-null    float64
 13  close_nextday    441 non-null    float64
dtypes: float64(14)
memory usage: 71.7 KB


In [45]:
# save the cleaned and compiled dataset

btc_df.to_csv('btc_prediction_datawranglingStage.csv')

# Conclusion

During this stage, we extracted the dataset of BTCUSD from the Binance exchanges API. Once the data was extracted we removed columns that are not considered in initial proposal, sticking to the data only contained in "openPrice", "high", "low", "closePrice". We used the timestamp as our index, prepping for a time series analysis and merging of additional datasets. Before joining the tweet sentiment analysis dataframe, we calculated the technical indicators measure for each day. The indicators used were, RSI, EMA_Long, EMA_Short, ATR, ADX and OBV. These indicators were calculated using the historical data pulled in from Binances API. After we calculated the technical indicators we brought in the tweet sentiment analysis dataframe. 

The tweet sentiment was derived from a open source webscraper utilizing historical tweets. The acquired tweets were cleaned and primed for joining to the main BTC dataframe. To get the sentiment we used a simple mean polarity calculation using TextBlobs library. See notebook "BTC_Tweet Scrape" for steps and processes. 

Once we joined the tweet sentiment dataframe, we also added an additional column containing the BTCUSDT Future prices for each day. That data was brought in from Binances API as well and cleaned in a seperate notebook. The idea of adding the BTC future prices to the dataframe is to see if they will be of any use in regards to trainable features for predicting prices. The next step is to do an Exploratory Data Analysis on the finalized and cleaned dataframe containing all the data we intend to use. Hopefully the EDA will reveal the potential use of each feature in ability to predict BTCs daily price. 